In [2]:
% pip install sklearn
% pip install pandas
% pip install numpy
% pip install yfinance
% pip install pandas_datareader

2023/01/13 09:38:56 INFO mlflow.recipes.recipe: Creating MLflow Recipe 'regression' with profile: 'local'


In [ ]:
import numpy as np
import datetime
import yfinance as yf
import pandas as pd
from sklearn.model_selection import train_test_split
from pandas_datareader import data as pdr

In [ ]:
def prepare_data(X, Y):
    X = pd.DataFrame(X)
    X.columns = ["day_" + str(i) for i in range(14)]
    Y = pd.DataFrame(Y)
    Y.columns = ["to_predict"]
    df = pd.concat([X, Y], axis=1)

    train_data, test_data = train_test_split(df, test_size=0.25, random_state=4284)
    return train_data, test_data

In [ ]:

def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)


In [ ]:
def digitize(n):
    if n > 0:
        return 1
    return 0


In [ ]:

def prepare_training_data(data):
    data["Delta"] = data["Close"] - data["Open"]
    data["to_predict"] = data["Delta"].apply(lambda d: digitize(d))
    return data

In [ ]:
def acquire_training_data():
    yf.pdr_override()
    y_symbols = ["BTC-USD"]

    startdate = datetime.datetime(2022, 1, 1)
    enddate = datetime.datetime(2022, 12, 31)
    df = pdr.get_data_yahoo(y_symbols, start=startdate, end=enddate)
    return df

In [4]:
training_data = acquire_training_data()
prepared_training_data_df = prepare_training_data(training_data)
btc_mat = prepared_training_data_df.to_numpy()
WINDOW_SIZE = 14
X = rolling_window(btc_mat[:, 7], WINDOW_SIZE)[:-1, :]
Y = prepared_training_data_df["to_predict"].to_numpy()[WINDOW_SIZE:]
train_data, test_data = prepare_data(X, Y)

2023/01/13 09:39:15 INFO mlflow.recipes.utils.execution: ingest, split: No changes. Skipping.


In [5]:
print(train_data)

2023/01/13 09:40:21 INFO mlflow.recipes.utils.execution: ingest, split, transform: No changes. Skipping.


Name,Type
tpep_pickup_datetime,datetime64[ns]
tpep_dropoff_datetime,datetime64[ns]
trip_distance,float64
fare_amount,float64
pickup_zip,int32
dropoff_zip,int32
Name,Type
f_00,float64
f_01,float64
f_02,float64


In [6]:
print(test_data)

2023/01/13 09:40:26 INFO mlflow.recipes.utils.execution: ingest, split, transform, train: No changes. Skipping.


Metric,training,validation
root_mean_squared_error,4.88902,3.92421
weighted_mean_squared_error,24.7796,17.6921
example_count,8051,959
max_error,254.516,52.7717
mean_absolute_error,1.65949,1.78055
mean_absolute_percentage_error,8.44266e+13,0.158525
mean_on_target,12.3947,13.1517
mean_squared_error,23.9026,15.3994
r2_score,0.79541,0.870413
score,0.79541,0.870413
